In [18]:
import cv2
from cv2 import cuda
from math import*
import numpy as np
from numba import jit,njit
import time
import pandas as pd

In [19]:
map = np.zeros((200,200))
map[:25][30:45] = 1


In [25]:
@jit(nopython = True)
def check_colision_goal (loc_map, goal_x, goal_y, current_x, current_y):

    if current_x >= goal_x:
        loc_x_check = 100 -(current_x - goal_x)
        if current_y >= goal_y:
            loc_y_check = 100 +(current_y - goal_y)
        else:
            loc_y_check = 100 -(goal_y - current_y)
           
    else:
        loc_x_check = 100 + (goal_x - current_x)
        if current_y >= goal_y:
            loc_y_check = 100 +(current_y - goal_y)
        else:
            loc_y_check = 100 -(goal_y - current_y)
            

    loc_x_check = int(loc_x_check)
    loc_y_check = int(loc_y_check)
    # dò sang bên phải 
    for cnt_right in range(loc_x_check,200-loc_x_check):
        if loc_map[loc_y_check][loc_x_check] == 0:
            break
    for cnt_left in range(loc_x_check,-1,-1):
        if loc_map[loc_y_check][loc_x_check] == 0:
            break
    if cnt_right > 190 and cnt_left < 10:
        colision = True
        x_out = 0
        y_out = 0
    else:
        colision = False
        if cnt_right <= cnt_left:
            x_out = goal_x + cnt_right
            y_out = goal_y
        else:
            x_out = goal_x - cnt_left
            y_out = goal_y
    return colision, x_out,y_out

@jit(nopython=True)
def create_local_map(glo_map,current_x, current_y):
    start_x = max(current_x - 100, 0)
    start_y = max(current_y - 100, 0)
    end_x = min(start_x + 200, glo_map.shape[1])
    end_y = min(start_y + 200, glo_map.shape[0])
    loc_map = glo_map[start_y:end_y, start_x:end_x]
    return loc_map,start_x,end_x,start_y,end_y

In [100]:

start = time.time()
check_colision_goal (loc_map = map, goal_x = 315, goal_y =400, current_x=400, current_y=315)

end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))



Elapsed (with compilation) = 0.0
Elapsed (with compilation) = 0.0


In [ ]:
# Tạo một hình ảnh trắng
image = 255 * np.ones((500, 500, 3), dtype=np.uint8)

# Tọa độ và kích thước của hình vuông
x, y = 100, 100
size = 200

# Màu sắc của hình vuông (BGR)
color = (0, 255, 0)  # Màu xanh lá cây

# Độ dày của đường viền (nếu muốn)
thickness = 2

# Vẽ hình vuông trên hình ảnh
cv2.rectangle(image, (x, y), (x + size, y + size), color, thickness)

# Hiển thị hình ảnh với hình vuông đã vẽ
cv2.imshow("Square", image)
cv2.waitKey(0)
cv2.destroyAllWindows()